In [ ]:
docker run -d --name my-container -e POSTGRES_PASSWORD=password -e POSTGRES_USER=postgres -e POSTGRES_DB=e-analytics_db -p 5432:5432 postgres:latest

docker start my-container

In [ ]:
jupyter nbconvert --to script example_notebook.ipynb --output ./scripts/example_script.py


In [2]:
# Iterate over all URLs to scrape data and save to a CSV file
for url in urls:
    # Get the last page number for the current URL
    last_page = get_last_page_number(url, headers)

    # Initialize an empty list to store all products
    products_list = []

    # Iterate through all pages from 1 to the last page
    for page_num in range(1, last_page + 1):
        page_url = f"{url}?page={page_num}#catalog-listing"
        print(f"Scraping page {page_num} from {url}...")
        
        # Scrape the products from the current page
        products = scrape_product_details(page_url, headers)
        products_list.extend(products)  # Add the scraped products to the main list

        # Sleep for a random time between requests to avoid overwhelming the server
        time.sleep(random.uniform(1, 3))

    # Determine the output CSV file name based on the URL
    if url == url1:
        csv_filename = 'data/scrapped/jumia_scraped_televisions.csv' 
    else:
        csv_filename = 'data/scrapped/jumia_scraped_cookers.csv'

    # Save the scraped product details to a CSV file
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=[ 
            'name', 'discounted_price', 'previous_price', 'discount_%', 'id', 'brand', 'rating', 'reviews_count', 'link'
        ])
        writer.writeheader()  # Write the header row
        
        for product in products_list:
            writer.writerow(product)

    print(f"Scraped {len(products_list)} products from {url} and saved them to '{csv_filename}'.")

NameError: name 'urls' is not defined

In [ ]:
# Function to scrape product details from a given URL
def scrape_product_details(url, headers):
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    products = soup.find_all('a', class_='core')

    product_details = []

    for product in products:
        # Extract product link (relative URL)
        link = product['href'] if 'href' in product.attrs else None
        # Complete the URL if the link is relative
        link = f"https://www.jumia.co.ke{link}" if link and link.startswith('/') else link
    
        # Extract product name from 'data-gtm-name' attribute
        name = product.get('data-gtm-name', "N/A")
        
        # Extract product price (actual price) from the 'prc' div
        price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
        
        old_price = product.find('div', class_='prc')['data-oprc']
        discount = product.find('div', class_='bdg _dsct').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
        
        # Extract item ID (from data-gtm-id attribute)
        item_id = product.get('data-gtm-id', "N/A")
        
        # Extract item brand (from data-gtm-brand attribute)
        item_brand = product.get('data-gtm-brand', "N/A")
        
        # Extract the stars rating (from the 'stars _m _al' or 'stars _s' class)
        stars_rating = product.find('div', class_='stars _m _al') or product.find('div', class_='stars _s')
        if stars_rating:
            rating = stars_rating.get_text(strip=True).split(" out of ")[0]  # Extract the rating value (e.g., "3.9")
        else:
            rating = "N/A"
        
        # Extract reviews count (from the 'rev' class or 'verified ratings' link)
        #reviews = product.find('div', class_='rev')
        #if reviews:
        #   reviews_count = reviews.get_text(strip=True).split('(')[-1].split(')')[0]  # Extract the review count (e.g., "798")
        #else:
        #   reviews_link = product.find('a', class_='-plxs _more')
        #   reviews_count = reviews_link.get_text(strip=True).split('(')[-1].split(')')[0] if reviews_link else "N/A"

        # Extract reviews count (from the 'rev' class)
        reviews = product.find('div', class_='rev')
        if reviews:
        # Use regular expression to find the number inside parentheses
            reviews_count = re.search(r'\((\d+)\)', reviews.get_text(strip=True))
            reviews_count = reviews_count.group(1) if reviews_count else "N/A"
        else:
            reviews_link = product.find('a', class_='-plxs _more')
            reviews_count = re.search(r'\((\d+)\)', reviews_link.get_text(strip=True))
            reviews_count = reviews_count.group(1) if reviews_count else "N/A"

        # Store all the extracted product details
        product_details.append({
            'name': name,
            'discounted_price': price,
            'previous_price': old_price,
            'discount_%': discount,
            'id': item_id,
            'brand': item_brand,
            'rating': rating,
            'reviews_count': reviews,
            'link': link,
        })
    
    return product_details


In [ ]:
 Function to fetch and scrape product details from a single page
def scrape_product_details(url, headers):
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')
    catalog_divs = soup.find_all('div', attrs={'data-catalog': 'true'})

    products = []
    for catalog_div in catalog_divs:
        product = catalog_div.find('a', class_='core')
        
        if product:
            name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
            price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
            old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
            discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
            
            rating = product.find('div', class_='rev')
            if rating:
                stars = rating.find('div', class_='stars _s').get_text(strip=True) if rating.find('div', class_='stars _s') else "N/A"
                reviews_count = rating.get_text(strip=True).split('(')[-1].strip(')') if '(' in rating.get_text() else "N/A"
            else:
                stars = "N/A"
                reviews_count = "N/A"

            item_id = product.get('data-gtm-id', "N/A")
            item_brand = product.get('data-gtm-brand', "N/A")
            product_url = product['href'] if product.has_attr('href') else "N/A"


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Function to extract old price and discount from a webpage
def extract_price_and_discount(url):
    # Send HTTP request to fetch the page content
    response = requests.get(url)
    
    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    old_price = None
    discount = None

    # Try to extract data based on the first format
    price_div_1 = soup.find('div', class_='-dif -i-ctr')
    if price_div_1:
        # Get the old price (which is the current price in this case)
        current_price = price_div_1.find('span', class_='-tal -gy5 -lthr -fs16 -pvxs -ubpt')
        discount_span = price_div_1.find('span', class_='bdg _dsct _dyn -mls')
        
        if current_price and discount_span:
            current_price_text = current_price.get_text().strip()
            discount_text = discount_span.get_text().strip()
            
            # Extract the old price directly (this is the discounted price)
            old_price = current_price_text
            discount = discount_text

    # Try to extract data based on the second format
    if not old_price:  # If not found in the first format, check the second format
        price_div_2 = soup.find('div', class_='s-prc-w')
        if price_div_2:
            old_price_tag = price_div_2.find('div', class_='old')
            discount_div = price_div_2.find('div', class_='bdg _dsct _sm')

            if old_price_tag and discount_div:
                old_price = old_price_tag.get_text().strip()
                discount = discount_div.get_text().strip()

    # Return the results
    return old_price, discount

# Example URL (replace with the actual URL of the webpage you want to scrape)
url = 'https://www.jumia.co.ke/televisions/#catalog-listing'

# Extract old price and discount
old_price, discount = extract_price_and_discount(url)

if old_price and discount:
    print(f"Old Price: {old_price}")
    print(f"Discount: {discount}")
else:
    print("Could not extract price and discount.")


del from el.ipynb

In [ ]:
# Function to scrape product details from a given URL
def scrape_product_details(url, headers):
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    products = soup.find_all('a', class_='core')

    product_details = []

    for product in products:
        # Extract product link (relative URL)
        link = product['href'] if 'href' in product.attrs else None
        # Complete the URL if the link is relative
        link = f"https://www.jumia.co.ke{link}" if link and link.startswith('/') else link
    
        name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
        price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
        old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
        discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
        
        rating = product.find('div', class_='rev')
        if rating:
            stars = rating.find('div', class_='stars _s').get_text(strip=True) if rating.find('div', class_='stars _s') else "N/A"
            reviews_count = rating.get_text(strip=True).split('(')[-1].strip(')') if '(' in rating.get_text() else "N/A"
        else:
            stars = "N/A"
            reviews_count = "N/A"
        
        item_id = product.get('data-gtm-id', "N/A")
        item_brand = product.get('data-gtm-brand', "N/A")
        product_url = product['href'] if product.has_attr('href') else "N/A"

                
        # Store all the extracted product details
        product_details.append({
            'name': name,
            'discounted_price': price,
            'previous_price': old_price,
            'discount_%': discount,
            'id': item_id,
            'brand': item_brand,
            'rating': rating,
            'reviews_count': reviews,
            'link': link,
        })


        # Optional delay between requests
        time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds
        
    return product_details


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import psycopg2

# Function to fetch and scrape product details from a single page
def scrape_product_details(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    catalog_divs = soup.find_all('div', attrs={'data-catalog': 'true'})

    products = []
    for catalog_div in catalog_divs:
        product = catalog_div.find('a', class_='core')
        
        if product:
            name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
            price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
            old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
            discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
            
            rating = product.find('div', class_='rev')
            if rating:
                stars = rating.find('div', class_='stars _s').get_text(strip=True) if rating.find('div', class_='stars _s') else "N/A"
                reviews_count = rating.get_text(strip=True).split('(')[-1].strip(')') if '(' in rating.get_text() else "N/A"
            else:
                stars = "N/A"
                reviews_count = "N/A"

            item_id = product.get('data-gtm-id', "N/A")
            item_brand = product.get('data-gtm-brand', "N/A")
            product_url = product['href'] if product.has_attr('href') else "N/A"

            # Store product info in a dictionary
            products.append({
                "product_name": name,
                "price": price,
                "old_price": old_price,
                "discount": discount,
                "rating": f"{stars} ({reviews_count} reviews)",
                "item_id": item_id,
                "item_brand": item_brand,
                "product_url": f"https://www.jumia.co.ke{product_url}"
            })
        
        # Optional delay between requests
        time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

    return products

# Function to get the last page number from the website
def get_last_page_number(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return 1  # Return 1 if failed to fetch page number

    soup = BeautifulSoup(response.text, 'html.parser')

    # Try to find the pagination section
    pagination = soup.find('ul', class_='pagination')

    # If no pagination is found, return 1 (assuming only one page)
    if pagination is None:
        print("No pagination found, assuming only one page.")
        return 1

    # If pagination is found, find the last page number
    try:
        last_page = pagination.find_all('li')[-1].get_text(strip=True)
        return int(last_page) if last_page.isdigit() else 1
    except Exception as e:
        print(f"Error parsing pagination: {e}")
        return 1

# Function to save data to PostgreSQL
def save_to_postgresql(products, db_params):
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Insert data into the database
        insert_query = """
        INSERT INTO jumia_televisions (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        for product in products:
            cursor.execute(insert_query, (
                product["product_name"],
                product["price"],
                product["old_price"],
                product["discount"],
                product["rating"],
                product["item_id"],
                product["item_brand"],
                product["product_url"]
            ))

        # Commit the transaction
        conn.commit()

        print(f"Successfully inserted {len(products)} products into the database.")

    except Exception as e:
        print(f"Error occurred: {e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()

# Database connection parameters
db_params = {
    "host": "localhost",     # Database host
    "database": "e-analytics_db",   # Database name
    "user": "postgres",     # Database user
    "password": "password"  # Database password
}

# Function to scrape and save data for multiple URLs
def scrape_and_save(urls, db_params):
    for url in urls:
        print(f"Scraping URL: {url}")
        
        # Get the last page number for the current URL
        last_page = get_last_page_number(url)

        # Initialize an empty list to store all products
        products_list = []

        # Iterate through all pages from 1 to the last page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num} from {url}...")

            # Scrape the products from the current page
            products = scrape_product_details(page_url)
            products_list.extend(products)  # Add the scraped products to the main list

            # Sleep for a random time between requests to avoid overwhelming the server
            time.sleep(random.uniform(1, 3))

        # Save the products to PostgreSQL after scraping all pages for this URL
        if products_list:
            save_to_postgresql(products_list, db_params)
            print(f"Saved {len(products_list)} products from {url} to PostgreSQL.")
        else:
            print(f"No products found on {url}.")

# Define your headers and the URLs to scrape
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List of URLs to scrape (example)
urls = [
    "https://www.jumia.co.ke/cookers/#catalog-listing",
    # Add more URLs as needed
]

# Start the main scraping process
scrape_and_save(urls, db_params)


In [ ]:
# Function to fetch and scrape product details from a single page
def scrape_product_details(url, headers):
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')
    catalog_divs = soup.find_all('div', attrs={'data-catalog': 'true'})

    products = []
    for catalog_div in catalog_divs:
        product = catalog_div.find('a', class_='core')
        
        if product:
            name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
            price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
            old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
            discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
            
            rating = product.find('div', class_='rev')
            if rating:
                stars = rating.find('div', class_='stars _s').get_text(strip=True) if rating.find('div', class_='stars _s') else "N/A"
                reviews_count = rating.get_text(strip=True).split('(')[-1].strip(')') if '(' in rating.get_text() else "N/A"
            else:
                stars = "N/A"
                reviews_count = "N/A"

            item_id = product.get('data-gtm-id', "N/A")
            item_brand = product.get('data-gtm-brand', "N/A")
            product_url = product['href'] if product.has_attr('href') else "N/A"

            # Store product info in a dictionary
            products.append({
                "product_name": name,
                "price": price,
                "old_price": old_price,
                "discount": discount,
                "rating": f"{stars} ({reviews_count} reviews)",
                "item_id": item_id,
                "item_brand": item_brand,
                "product_url": f"https://www.jumia.co.ke{product_url}"
            })
        
        # Optional delay between requests
        time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

    return products

# Function to get the last page number from the website using the 'Last Page' link
def get_last_page_number(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the 'Last Page' link using its aria-label attribute
    last_page_link = soup.find('a', attrs={'aria-label': 'Last Page'})
    
    if last_page_link and 'href' in last_page_link.attrs:
        last_page_url = last_page_link['href']
        try:
            page_number = last_page_url.split('?page=')[1].split('#')[0]
            return int(page_number)
        except Exception as e:
            print(f"Error extracting last page number: {e}")
            return 1  # Default to 1 if error occurs
    else:
        print("Last page link not found.")
        return 1  # Default to 1 page if no last page is found

# Function to save data to PostgreSQL
def save_to_postgresql(products, db_params):
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Insert data into the database
        insert_query = """
        INSERT INTO jumia_televisions (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        for product in products:
            cursor.execute(insert_query, (
                product["product_name"],
                product["price"],
                product["old_price"],
                product["discount"],
                product["rating"],
                product["item_id"],
                product["item_brand"],
                product["product_url"]
            ))

        # Commit the transaction
        conn.commit()

        print(f"Successfully inserted {len(products)} products into the database.")

    except Exception as e:
        print(f"Error occurred: {e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()

# Database connection parameters
db_params = {
    "host": "localhost",     # Database host
    "database": "e-analytics_db",   # Database name
    "user": "postgres",     # Database user
    "password": "password"  # Database password
}

# Function to scrape and save data for multiple URLs
def scrape_and_save(urls, headers, db_params):
    for url in urls:
        print(f"Scraping URL: {url}")
        
        # Get the last page number for the current URL
        last_page = get_last_page_number(url, headers)

        # Initialize an empty list to store all products
        products_list = []

        # Iterate through all pages from 1 to the last page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num} from {url}...")

            # Scrape the products from the current page
            products = scrape_product_details(page_url, headers)
            products_list.extend(products)  # Add the scraped products to the main list

            # Sleep for a random time between requests to avoid overwhelming the server
            time.sleep(random.uniform(1, 3))

        # Save the products to PostgreSQL after scraping all pages for this URL
        if products_list:
            save_to_postgresql(products_list, db_params)
            print(f"Saved {len(products_list)} products from {url} to PostgreSQL.")
        else:
            print(f"No products found on {url}.")

# Define your headers and the URLs to scrape
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List of URLs to scrape (example)
urls = [
    url1,
    url2,
    # Add more URLs as needed
]

# Start the main scraping process
scrape_and_save(urls, headers, db_params)


In [ ]:
# Function to get the last page number from the website
def get_last_page_number(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Try to find the pagination section
    pagination = soup.find('ul', class_='pagination')

    # If no pagination is found, return 1 (assuming only one page)
    if pagination is None:
        print("No pagination found, assuming only one page.")
        return 1

    # If pagination is found, find the last page number
    try:
        last_page = pagination.find_all('li')[-1].get_text(strip=True)
        return int(last_page) if last_page.isdigit() else 1
    except Exception as e:
        print(f"Error parsing pagination: {e}")
        return 1

# Function to save data to PostgreSQL
def save_to_postgresql(products, db_params):
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Insert data into the database
        insert_query = """
        INSERT INTO jumia_televisions (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        for product in products:
            cursor.execute(insert_query, (
                product["product_name"],
                product["price"],
                product["old_price"],
                product["discount"],
                product["rating"],
                product["item_id"],
                product["item_brand"],
                product["product_url"]
            ))

        # Commit the transaction
        conn.commit()

        print(f"Successfully inserted {len(products)} products into the database.")

    except Exception as e:
        print(f"Error occurred: {e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()

# Database connection parameters
db_params = {
    "host": "localhost",     # Database host
    "database": "e-analytics_db",   # Database name
    "user": "postgres",     # Database user
    "password": "password"  # Database password
}

# Function to scrape and save data for multiple URLs
def scrape_and_save(urls, headers, db_params):
    for url in urls:
        print(f"Scraping URL: {url}")
        
        # Get the last page number for the current URL
        last_page = get_last_page_number(url, headers)

        # Initialize an empty list to store all products
        products_list = []

        # Iterate through all pages from 1 to the last page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num} from {url}...")

            # Scrape the products from the current page
            products = scrape_product_details(page_url, headers)
            products_list.extend(products)  # Add the scraped products to the main list

            # Sleep for a random time between requests to avoid overwhelming the server
            time.sleep(random.uniform(1, 3))

        # Save the products to PostgreSQL after scraping all pages for this URL
        if products_list:
            save_to_postgresql(products_list, db_params)
            print(f"Saved {len(products_list)} products from {url} to PostgreSQL.")
        else:
            print(f"No products found on {url}.")

# Define your headers and the URLs to scrape
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List of URLs to scrape (example)
urls = [
    url1,
    url2,
    # Add more URLs as needed
]

# Start the main scraping process
scrape_and_save(urls, headers, db_params)


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import psycopg2

# Function to fetch and scrape product details
def scrape_product_details(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    catalog_divs = soup.find_all('div', attrs={'data-catalog': 'true'})

    products = []
    for catalog_div in catalog_divs:
        product = catalog_div.find('a', class_='core')
        
        if product:
            name = product.find('h3', class_='name').get_text(strip=True) if product.find('h3', class_='name') else "N/A"
            price = product.find('div', class_='prc').get_text(strip=True) if product.find('div', class_='prc') else "N/A"
            old_price = product.find('div', class_='old').get_text(strip=True) if product.find('div', class_='old') else "N/A"
            discount = product.find('div', class_='bdg _dsct _sm').get_text(strip=True) if product.find('div', class_='bdg _dsct _sm') else "N/A"
            
            rating = product.find('div', class_='rev')
            if rating:
                stars = rating.find('div', class_='stars _s').get_text(strip=True) if rating.find('div', class_='stars _s') else "N/A"
                reviews_count = rating.get_text(strip=True).split('(')[-1].strip(')') if '(' in rating.get_text() else "N/A"
            else:
                stars = "N/A"
                reviews_count = "N/A"

            item_id = product.get('data-gtm-id', "N/A")
            item_brand = product.get('data-gtm-brand', "N/A")
            product_url = product['href'] if product.has_attr('href') else "N/A"

            # Store product info in a dictionary
            products.append({
                "product_name": name,
                "price": price,
                "old_price": old_price,
                "discount": discount,
                "rating": f"{stars} ({reviews_count} reviews)",
                "item_id": item_id,
                "item_brand": item_brand,
                "product_url": f"https://www.jumia.co.ke{product_url}"
            })
        
        # Optional delay between requests
        time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

    return products

# Function to get the last page number from the website
def get_last_page_number(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return 1  # Return 1 if failed to fetch page number

    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the last page number (you might need to adjust the selector based on the website's structure)
    pagination = soup.find('ul', class_='pagination')
    last_page = pagination.find_all('li')[-1].get_text(strip=True)
    return int(last_page) if last_page.isdigit() else 1

# Function to save data to PostgreSQL
def save_to_postgresql(products, db_params):
    try:
        # Connect to your PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Insert data into the database
        insert_query = """
        INSERT INTO jumia_cookers (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """

        for product in products:
            cursor.execute(insert_query, (
                product["product_name"],
                product["price"],
                product["old_price"],
                product["discount"],
                product["rating"],
                product["item_id"],
                product["item_brand"],
                product["product_url"]
            ))

        # Commit the transaction
        conn.commit()

        print(f"Successfully inserted {len(products)} products into the database.")

    except Exception as e:
        print(f"Error occurred: {e}")
        conn.rollback()

    finally:
        cursor.close()
        conn.close()

# Database connection parameters
db_params = {
    "host": "localhost",     # Database host
    "database": "e-analytics_db",   # Database name
    "user": "postgres",     # Database user
    "password": "password"  # Database password
}

# Function to scrape and save data for multiple URLs
def scrape_and_save(urls, headers, db_params):
    for url in urls:
        print(f"Scraping URL: {url}")
        # Get the last page number for the current URL
        last_page = get_last_page_number(url, headers)

        # Initialize an empty list to store all products
        products_list = []

        # Iterate through all pages from 1 to the last page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num} from {url}...")

            # Scrape the products from the current page
            products = scrape_product_details(page_url, headers)
            products_list.extend(products)  # Add the scraped products to the main list

            # Sleep for a random time between requests to avoid overwhelming the server
            time.sleep(random.uniform(1, 3))

        # Save the products to PostgreSQL after scraping all pages for this URL
        if products_list:
            save_to_postgresql(products_list, db_params)
            print(f"Saved {len(products_list)} products from {url} to PostgreSQL.")
        else:
            print(f"No products found on {url}.")

# Define your headers and the URLs to scrape
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List of URLs to scrape (example)
urls = [
    "https://www.jumia.co.ke/home-cooking-appliances-cookers/#catalog-listing"
    # Add more URLs as needed
]

# Start the main scraping process
scrape_and_save(urls, headers, db_params)
